# fecundity.ipynb

In [4]:
!pip install sqlalchemy

     |████████████████████████████████| 3.1 MB 14 kB/s eta 0:00:011
     |████████████████████████████████| 667 kB 46 kB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1


In [8]:
!pip install pandas

     |████████████████████████████████| 12.4 MB 666 kB/s eta 0:00:01
     |████████████████████████████████| 345 kB 1.0 MB/s eta 0:00:01


In [18]:
!pip install mysql-connector-python

     |████████████████████████████████| 557 kB 682 kB/s eta 0:00:01


In [19]:
from sqlalchemy import create_engine, text
import getpass
import mysql.connector as mysql
from datetime import datetime
import pandas as pd

In [20]:
def get_database_engine():
  username = input('Enter username:')
  print('Enter password:')
  password = getpass.getpass()

  host = 'mysql.guaminsects.net'
  port = '3306'
  database = 'ornv'
  connection_string = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'
  engine = create_engine(connection_string)
  return engine

engine = get_database_engine()

Enter password:


In [24]:
def get_bioassay_data(date_dosed):
  q = f"""
  SELECT sn, date_dosed, treatment, date_dead
  FROM beetles
  WHERE date_dosed = '{date_dosed}'
  """
  df = pd.read_sql_query(sql=text(q), con=engine.connect())
  # Add a few calculated fields
  # df['days_post_treatment'] = (df['date_dead'] - df['date_dosed']).dt.days
  # df['dead'] = df['days_post_treatment'].notna()
  # df['days_post_treatment'] = df.days_post_treatment.fillna(df.days_post_treatment.max())
  return df

df = get_bioassay_data('2023-08-15')

In [27]:
df = pd.read_sql_table('beetles', engine.connect(), parse_dates=['date_collected', 'date_dosed', 'date_dead'])
df

,sn,date_collected,trap_id,sex,elytra_length,elytra_width,date_dosed,treatment,dose_mg,date_dead,pm_mites,pm_nema,pm_heartbeat,pm_image,pm_sample_id_prefix,pm_sample_id,notes
0,1,2023-07-09,1,f,20.68,17.90,NaT,,0,NaT,0,0,0,0,,0,"original_feeding_group: Monday, used_for: , fe..."
1,2,2023-07-09,1,m,21.36,17.57,2023-08-24,CONTROL,94,2023-08-29,0,0,0,0,,0,"original_feeding_group: Monday, used_for: , fe..."
2,3,2023-07-09,4,f,25.46,20.06,NaT,,0,2023-08-14,0,0,0,0,,0,"original_feeding_group: Monday, used_for: , fe..."
3,4,2023-07-09,4,m,26.69,20.92,2023-08-24,PNG,175,2023-09-21,0,1,0,1,GUAM 2023,21,"original_feeding_group: Monday, used_for: , fe..."
4,5,2023-07-09,5,f,22.95,19.14,NaT,,0,2023-08-10,0,0,0,0,,0,"original_feeding_group: Monday, used_for: , fe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,483,2023-08-23,39,f,24.01,18.20,NaT,,0,NaT,0,0,0,0,,0,"original_feeding_group: Thursday, used_for: , ..."
482,484,2023-08-23,39,f,25.08,20.07,NaT,,0,NaT,0,0,0,0,,0,"original_feeding_group: Thursday, used_for: , ..."
483,485,2023-08-23,35,m,23.23,17.19,NaT,,0,NaT,0,0,0,0,,0,"original_feeding_group: Thursday, used_for: , ..."
484,486,2023-08-23,40,f,22.31,17.54,NaT,,0,NaT,0,0,0,0,,0,"original_feeding_group: Thursday, used_for: , ..."


In [29]:
df.to_csv('beetles.csv', index=False)

In [104]:
df_beetles = pd.read_csv('beetles.csv', parse_dates=['date_collected', 'date_dosed', 'date_dead'])
df_beetles['days_between_treatment_and_death'] = df_beetles.date_dead - df_beetles.date_dosed
df_beetles

,sn,date_collected,trap_id,sex,elytra_length,elytra_width,date_dosed,treatment,dose_mg,date_dead,pm_mites,pm_nema,pm_heartbeat,pm_image,pm_sample_id_prefix,pm_sample_id,notes,days_between_treatment_and_death
0,1,2023-07-09,1,f,20.68,17.90,NaT,NaN,0,NaT,0,0,0,0,NaN,0,"original_feeding_group: Monday, used_for: , fe...",NaT
1,2,2023-07-09,1,m,21.36,17.57,2023-08-24,CONTROL,94,2023-08-29,0,0,0,0,NaN,0,"original_feeding_group: Monday, used_for: , fe...",5 days
2,3,2023-07-09,4,f,25.46,20.06,NaT,NaN,0,2023-08-14,0,0,0,0,NaN,0,"original_feeding_group: Monday, used_for: , fe...",NaT
3,4,2023-07-09,4,m,26.69,20.92,2023-08-24,PNG,175,2023-09-21,0,1,0,1,GUAM 2023,21,"original_feeding_group: Monday, used_for: , fe...",28 days
4,5,2023-07-09,5,f,22.95,19.14,NaT,NaN,0,2023-08-10,0,0,0,0,NaN,0,"original_feeding_group: Monday, used_for: , fe...",NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,483,2023-08-23,39,f,24.01,18.20,NaT,NaN,0,NaT,0,0,0,0,NaN,0,"original_feeding_group: Thursday, used_for: , ...",NaT
482,484,2023-08-23,39,f,25.08,20.07,NaT,NaN,0,NaT,0,0,0,0,NaN,0,"original_feeding_group: Thursday, used_for: , ...",NaT
483,485,2023-08-23,35,m,23.23,17.19,NaT,NaN,0,NaT,0,0,0,0,NaN,0,"original_feeding_group: Thursday, used_for: , ...",NaT
484,486,2023-08-23,40,f,22.31,17.54,NaT,NaN,0,NaT,0,0,0,0,NaN,0,"original_feeding_group: Thursday, used_for: , ...",NaT


In [41]:
# df = pd.read_sql_table('mass', engine.connect(), parse_dates=['dt'])
# df.to_csv('mass.csv', index=False)
df_mass = pd.read_csv('mass.csv', parse_dates=(['dt']))
df_mass

,mass_id,sn,dt,mg,note
0,1,35,2023-08-04 12:00:00,3011,NaN
1,2,69,2023-08-04 12:00:00,5263,NaN
2,3,46,2023-08-04 12:00:00,3900,NaN
3,4,80,2023-08-04 12:00:00,5286,NaN
4,5,32,2023-08-04 12:00:00,4459,NaN
...,...,...,...,...,...
765,951,451,2023-08-31 12:01:00,4325,NaN
766,952,454,2023-08-31 12:00:00,5268,NaN
767,953,454,2023-08-31 12:01:00,5296,NaN
768,954,466,2023-08-31 12:00:00,3193,NaN


In [52]:
df_mass.query('sn == 445')

,mass_id,sn,dt,mg,note
758,944,445,2023-08-31 12:00:00,3517,NaN
759,945,445,2023-08-31 12:01:00,3557,NaN


In [76]:
df = df_beetles.query('date_dosed == "2023-08-24"')
df = df.query(f'date_dead != "{df.date_dead.max()}"')
df              

,sn,date_collected,trap_id,sex,elytra_length,elytra_width,date_dosed,treatment,dose_mg,date_dead,pm_mites,pm_nema,pm_heartbeat,pm_image,pm_sample_id_prefix,pm_sample_id,notes
1,2,2023-07-09,1,m,21.36,17.57,2023-08-24,CONTROL,94,2023-08-29,0,0,0,0,NaN,0,"original_feeding_group: Monday, used_for: , fe..."
3,4,2023-07-09,4,m,26.69,20.92,2023-08-24,PNG,175,2023-09-21,0,1,0,1,GUAM 2023,21,"original_feeding_group: Monday, used_for: , fe..."
6,7,2023-07-09,33,m,18.86,15.57,2023-08-24,PNG,76,2023-10-02,1,1,0,1,GUAM 2023,46,"original_feeding_group: Monday, used_for: , fe..."
10,11,2023-07-12,1,m,24.65,20.11,2023-08-24,CONTROL,66,2023-10-17,0,1,1,1,GUAM 202309,83,"original_feeding_group: Monday, used_for: , fe..."
35,36,2023-07-14,40,m,23.54,17.69,2023-08-24,CONTROL,111,2023-09-19,1,1,1,1,GUAM 2023,18,"original_feeding_group: Thursday, used_for: , ..."
39,40,2023-07-17,40,m,24.56,19.75,2023-08-24,V23B,71,2023-08-29,0,1,0,0,AgR Team 202209,20,"original_feeding_group: Monday, used_for: , fe..."
64,65,2023-07-17,20,f,22.45,17.33,2023-08-24,V23B,44,2023-10-11,0,1,1,1,GUAM 202309,63,"original_feeding_group: Thursday, used_for: , ..."
65,66,2023-07-17,4,f,23.93,19.54,2023-08-24,V23B,35,2023-09-04,0,1,0,1,AgR Team 202209,42,yellowish ooze on tip of abdomen
66,67,2023-07-17,1,f,23.94,18.84,2023-08-24,X2B,81,2023-08-28,0,0,0,0,AgR Team 202209,18,"original_feeding_group: Thursday, used_for: , ..."
74,75,2023-07-20,4,m,24.30,19.68,2023-08-24,DUG42,196,2023-09-10,1,1,1,1,none,58,many nematodes beneeath elytra


In [68]:
sorted(df_beetles.date_dosed.unique())

[NaT,
 Timestamp('2023-07-31 00:00:00'),
 Timestamp('2023-08-04 00:00:00'),
 Timestamp('2023-08-15 00:00:00'),
 Timestamp('2023-08-24 00:00:00'),
 Timestamp('2023-08-31 00:00:00')]

In [96]:
dt = df.date_dead - df.date_dosed
dt

1      5 days
3     28 days
6     39 days
10    54 days
35    26 days
39     5 days
64    48 days
65    11 days
66     4 days
74    17 days
80    32 days
90    43 days
94    10 days
95    46 days
100       NaT
103   48 days
105   26 days
106   53 days
108    8 days
110   49 days
115   19 days
117    2 days
122   21 days
134   39 days
137   12 days
142   53 days
151   11 days
152   48 days
153    4 days
157   31 days
158   31 days
163   21 days
169   10 days
173   50 days
175   53 days
181   35 days
182   31 days
183   25 days
187   35 days
189   21 days
194    1 days
195   53 days
201   10 days
206   54 days
207   54 days
209   21 days
210   40 days
214   39 days
261   35 days
271    7 days
dtype: timedelta64[ns]

In [101]:
dt.median()

Timedelta('31 days 00:00:00')